## Accuracy

In [12]:
import pandas as pd
import re
import os

def parse_training_results(file_path):
    """Parse training results from text file into a pandas DataFrame"""
    
    # Read the file
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Initialize lists to store data
    epochs = []
    times = []
    train_losses = []
    train_acc_top1 = []
    train_acc_top5 = []
    test_losses = []
    test_acc_top1 = []
    test_acc_top5 = []
    
    # Regular expression to match each epoch line
    pattern = r'\[Epoch (\d+)\] Time: ([\d.]+)s.*?Train.*?Loss: ([\d.]+).*?Top1: ([\d.]+)%.*?Top5: ([\d.]+)%.*?Test.*?Loss: ([\d.]+).*?Top1: ([\d.]+)%.*?Top5: ([\d.]+)%'
    
    # Find all matches
    matches = re.findall(pattern, content)
    
    for match in matches:
        epochs.append(int(match[0]))
        times.append(float(match[1]))
        train_losses.append(float(match[2]))
        train_acc_top1.append(float(match[3]))
        train_acc_top5.append(float(match[4]))
        test_losses.append(float(match[5]))
        test_acc_top1.append(float(match[6]))
        test_acc_top5.append(float(match[7]))
    
    # Create DataFrame
    df = pd.DataFrame({
        'epoch': epochs,
        'time': times,
        'train_loss': train_losses,
        'train_accuracy_top1': train_acc_top1,
        'train_accuracy_top5': train_acc_top5,
        'test_loss': test_losses,
        'test_accuracy_top1': test_acc_top1,
        'test_accuracy_top5': test_acc_top5
    })
    
    return df


def n_last_average_stats(df, last_n_epochs=5):
    """Calculate average stats over the last N epochs"""
    if len(df) < last_n_epochs:
        last_n_epochs = len(df)
    
    return {
        "ave_train_loss": df['train_loss'].tail(last_n_epochs).mean(),
        "ave_test_loss": df['test_loss'].tail(last_n_epochs).mean(),
        "ave_train_accuracy_top1": df['train_accuracy_top1'].tail(last_n_epochs).mean(),
        "ave_train_accuracy_top5": df['train_accuracy_top5'].tail(last_n_epochs).mean(),
        "ave_test_accuracy_top1": df['test_accuracy_top1'].tail(last_n_epochs).mean(),
        "ave_test_accuracy_top5": df['test_accuracy_top5'].tail(last_n_epochs).mean()
    }

def best_stats(df):
    """Calculate best accuracy achieved during training"""
    return {
        "best_train_accuracy_top1": df['train_accuracy_top1'].max(),
        "best_train_accuracy_top5": df['train_accuracy_top5'].max(),
        "best_test_accuracy_top1": df['test_accuracy_top1'].max(),
        "best_test_accuracy_top5": df['test_accuracy_top5'].max()
    }

def parse_params_gflops(file_dir):
    """Parse training results from text file into a pandas DataFrame"""

    train_path = os.path.join(file_dir, "train_eval_results.txt")
    args_path = os.path.join(file_dir, "args.txt")
    
    # Read the file
    with open(train_path, 'r') as f:
        train_content = f.readline()

    with open(args_path, 'r') as f:
        args_content = f.readlines()

    total_params = args_content[-2].strip().split(": ")[1]
    trainable_params = args_content[-1].strip().split(": ")[1]

    gflops = re.search(r'GFLOPs: ([\d.]+)', train_content).group(1)

    return {
        "total_params": int(total_params),
        "trainable_params": int(trainable_params),
        "gflops": float(gflops)
    }



In [13]:
import sys 
import os
import torch 
import torch.nn as nn

curr_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(curr_dir, '..', '..'))

print("Current Directory:", curr_dir)
print("Root Directory:", root_dir)

Current Directory: /Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/N_test_cos
Root Directory: /Users/mingikang/Developer/Convolutional-Nearest-Neighbor


In [ ]:
datasets = ["VGG11-CIFAR10", "VGG11-CIFAR100"]
bratio = ["1000", "0500"]
type = ["random", "spatial"]
Ns = ["4", "6", "8", "10", "12", "14", "16", "18", "20", "22", "24", "26", "28", "30", "32"]
rows = []
for ds in datasets: 
    for br in bratio:
        for t in type:
            for n in Ns:

                model_name = f"BranchingConvNN_{t}_NS{n}_col_col_br{br}_s42"
                model_path = os.path.join(curr_dir, ds, model_name, "train_eval_results.txt")


                df = parse_training_results(model_path)
                avg_stats = n_last_average_stats(df, last_n_epochs=5)
                best_acc = best_stats(df)

                gflops_path = os.path.join(curr_dir, ds, model_name)
                gflops = parse_params_gflops(gflops_path)
                
                if br == "1000":
                    model = "ConvNN"
                else: 
                    model = "Branching"
    
                rows.append({
                    "Model": model, 
                    "Dataset": ds,
                    "Type": t,
                    "N": int(n),
                    "BestAcc": best_acc["best_test_accuracy_top1"],
                    "AvgAcc": avg_stats["ave_test_accuracy_top1"],
                    "TotalParams": gflops["total_params"],
                    "TrainableParams": gflops["trainable_params"],
                    "GFLOPs": gflops["gflops"]
                })


conv_cifar10 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR10/lr_1e-4/BranchingConvNN_K9_col_col_br0000_s42/train_eval_results.txt"

conv_cifar100 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR100/lr_1e-4/BranchingConvNN_K9_col_col_br0000_s42/train_eval_results.txt"

branching_cifar10 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR10/lr_1e-4/BranchingConvNN_K9_col_col_br0500_s42/train_eval_results.txt"
branching_cifar100 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR100/lr_1e-4/BranchingConvNN_K9_col_col_br0500_s42/train_eval_results.txt"

convnn_cifar10 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR10/lr_1e-4/BranchingConvNN_K9_col_col_br1000_s42/train_eval_results.txt"

convnn_cifar100 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR100/lr_1e-4/BranchingConvNN_K9_col_col_br1000_s42/train_eval_results.txt"

conv_cifar10_df = parse_training_results(conv_cifar10)
conv_cifar100_df = parse_training_results(conv_cifar100)
branching_cifar10_df = parse_training_results(branching_cifar10)
branching_cifar100_df = parse_training_results(branching_cifar100)
convnn_cifar10_df = parse_training_results(convnn_cifar10)
convnn_cifar100_df = parse_training_results(convnn_cifar100)

conv_cifar10_best = best_stats(conv_cifar10_df)
conv_cifar100_best = best_stats(conv_cifar100_df)
branching_cifar10_best = best_stats(branching_cifar10_df)
branching_cifar100_best = best_stats(branching_cifar100_df)
convnn_cifar10_best = best_stats(convnn_cifar10_df)
convnn_cifar100_best = best_stats(convnn_cifar100_df)


conv_cifar10_avg = n_last_average_stats(conv_cifar10_df, last_n_epochs=5)
conv_cifar100_avg = n_last_average_stats(conv_cifar100_df, last_n_epochs=5)
branching_cifar10_avg = n_last_average_stats(branching_cifar10_df, last_n_epochs=5)
branching_cifar100_avg = n_last_average_stats(branching_cifar100_df, last_n_epochs=5)
convnn_cifar10_avg = n_last_average_stats(convnn_cifar10_df, last_n_epochs=5)
convnn_cifar100_avg = n_last_average_stats(convnn_cifar100_df, last_n_epochs=5)


# Gflops 

conv_cifar10 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR10/lr_1e-4/BranchingConvNN_K9_col_col_br0000_s42/"

conv_cifar100 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR100/lr_1e-4/BranchingConvNN_K9_col_col_br0000_s42/"

branching_cifar10 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR10/lr_1e-4/BranchingConvNN_K9_col_col_br0500_s42/"
branching_cifar100 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR100/lr_1e-4/BranchingConvNN_K9_col_col_br0500_s42/"

convnn_cifar10 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR10/lr_1e-4/BranchingConvNN_K9_col_col_br1000_s42/"

convnn_cifar100 = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/loss_test_cos_first**/VGG11-CIFAR100/lr_1e-4/BranchingConvNN_K9_col_col_br1000_s42/"

conv_cifar10_gflops = parse_params_gflops(conv_cifar10)
conv_cifar100_gflops = parse_params_gflops(conv_cifar100)
branching_cifar10_gflops = parse_params_gflops(branching_cifar10)
branching_cifar100_gflops = parse_params_gflops(branching_cifar100)
convnn_cifar10_gflops = parse_params_gflops(convnn_cifar10)
convnn_cifar100_gflops = parse_params_gflops(convnn_cifar100)


rows.append({
    "Model": "Convolution", 
    "Dataset": "VGG11-CIFAR10",
    "Type": "baseline",
    "N": 0,
    "BestAcc": conv_cifar10_best["best_test_accuracy_top1"],
    "AvgAcc": conv_cifar10_avg["ave_test_accuracy_top1"],
    "TotalParams": conv_cifar10_gflops["total_params"],
    "TrainableParams": conv_cifar10_gflops["trainable_params"],
    "GFLOPs": conv_cifar10_gflops["gflops"]
})

rows.append({
    "Model": "Convolution", 
    "Dataset": "VGG11-CIFAR100",
    "Type": "baseline",
    "N": 0,
    "BestAcc": conv_cifar100_best["best_test_accuracy_top1"],
    "AvgAcc": conv_cifar100_avg["ave_test_accuracy_top1"],
    "TotalParams": conv_cifar100_gflops["total_params"],
    "TrainableParams": conv_cifar100_gflops["trainable_params"],
    "GFLOPs": conv_cifar100_gflops["gflops"]
})


rows.append({
    "Model": "ConvNN", 
    "Dataset": "VGG11-CIFAR10",
    "Type": "all",
    "N": -1,
    "BestAcc": convnn_cifar10_best["best_test_accuracy_top1"],
    "AvgAcc": convnn_cifar10_avg["ave_test_accuracy_top1"],
    "TotalParams": convnn_cifar10_gflops["total_params"],
    "TrainableParams": convnn_cifar10_gflops["trainable_params"],
    "GFLOPs": convnn_cifar10_gflops["gflops"]
})

rows.append({
    "Model": "ConvNN", 
    "Dataset": "VGG11-CIFAR100",
    "Type": "all",
    "N": -1,
    "BestAcc": convnn_cifar100_best["best_test_accuracy_top1"],
    "AvgAcc": convnn_cifar100_avg["ave_test_accuracy_top1"],
    "TotalParams": convnn_cifar100_gflops["total_params"],
    "TrainableParams": convnn_cifar100_gflops["trainable_params"],
    "GFLOPs": convnn_cifar100_gflops["gflops"]
})


rows.append({
    "Model": "Branching", 
    "Dataset": "VGG11-CIFAR10",
    "Type": "all",
    "N": -1,
    "BestAcc": branching_cifar10_best["best_test_accuracy_top1"],
    "AvgAcc": branching_cifar10_avg["ave_test_accuracy_top1"],
    "TotalParams": branching_cifar10_gflops["total_params"],
    "TrainableParams": branching_cifar10_gflops["trainable_params"],
    "GFLOPs": branching_cifar10_gflops["gflops"]
})

rows.append({
    "Model": "Branching", 
    "Dataset": "VGG11-CIFAR100",
    "Type": "all",
    "N": -1,
    "BestAcc": branching_cifar100_best["best_test_accuracy_top1"],
    "AvgAcc": branching_cifar100_avg["ave_test_accuracy_top1"],
    "TotalParams": branching_cifar100_gflops["total_params"],
    "TrainableParams": branching_cifar100_gflops["trainable_params"],
    "GFLOPs": branching_cifar100_gflops["gflops"]
})


results_df = pd.DataFrame(rows)
results_df.to_csv(os.path.join(curr_dir, "N_test_cos_results.csv"), index=False)
print(results_df.head())



    Model        Dataset    Type   N  BestAcc    AvgAcc  TotalParams  \
0  ConvNN  VGG11-CIFAR10  random   4  59.2969  58.07030    130015690   
1  ConvNN  VGG11-CIFAR10  random   6  57.6074  57.11914    130015690   
2  ConvNN  VGG11-CIFAR10  random   8  57.4805  56.76760    130015690   
3  ConvNN  VGG11-CIFAR10  random  10  57.5098  56.29102    130015690   
4  ConvNN  VGG11-CIFAR10  random  12  57.0020  56.33398    130015690   

   TrainableParams    GFLOPs  
0        130015690  0.297096  
1        130015690  0.300706  
2        130015690  0.304211  
3        130015690  0.308719  
4        130015690  0.310849  


In [9]:
base_path = "/Users/mingikang/Developer/Convolutional-Nearest-Neighbor/Final_Output/N_test_cos/"


# os.joinpath(base_path, dataset, f"BranchingConvNN_random_NS{N}_col_col_br{b}_s42", "training_results.txt")

## Gflops

In [12]:
def parse_params_gflops(file_dir):
    """Parse training results from text file into a pandas DataFrame"""

    train_path = os.path.join(file_dir, "train_eval_results.txt")
    args_path = os.path.join(file_dir, "args.txt")
    
    # Read the file
    with open(train_path, 'r') as f:
        train_content = f.readline()

    with open(args_path, 'r') as f:
        args_content = f.readlines()

    total_params = args_content[-2].strip().split(": ")[1]
    trainable_params = args_content[-1].strip().split(": ")[1]

    gflops = re.search(r'GFLOPs: ([\d.]+)', train_content).group(1)

    return {
        "total_params": int(total_params),
        "trainable_params": int(trainable_params),
        "gflops": float(gflops)
    }